In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [20]:
SVC = pd.read_csv('{파일경로}/SVC.csv')
TabNet = pd.read_csv('{파일경로}/TabNet.csv')
XGB = pd.read_csv('{파일경로}/XGB.csv')
RF = pd.read_csv('{파일경로}/RandomForest.csv')
test = pd.read_csv('{파일경로}/y_test.csv')

In [9]:
print(SVC.shape, TabNet.shape, XGB.shape, RF.shape, test.shape)

(105, 2) (105, 2) (105, 2) (105, 2) (105, 2)


In [10]:
# 데이터의 인덱스가 중복으로 들어가 'Unnamed: 0' 컬럼열 삭제
def drop_columns(df):
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
    return df

df_list = [SVC, TabNet, XGB, RF, test] # 해당 데이터 리스트

for df in df_list:
    drop_columns(df)
df = pd.concat([SVC, XGB, RF, TabNet, test], axis=1)

In [16]:
# 하이퍼파라미터
random_state = 8888 # 랜덤 시드 설정
n_est = [50] # 결정 트리 개수 설정
max_d = [None] # 결정 트리의 최대 깊이
max_f = [4] # 최대 특성 개수
min_s_s = [2] # 최소 분리 샘플 수 
min_s_l = [2] # 최소 잎새 샘플 수
c = ['gini'] # 결정 트리 분리 기준 
test_accuracy_list = [] # 각 반복에서 테스트 셋의 정확도를 저장할 리스트
results = [] # 모든 반복에서 테스트 셋 정화도를 저장할 리스트

smote = SMOTE() # 오버 샘플링 기법 중 SMOTE 사용

X, y = df.iloc[:, :-1], df.iloc[:, -1]
# X : pmStress 컬럼을 제외한 데이터 
# y : 'pmStress'

for i in range(20):
    train = pd.DataFrame() # train data를 저장할 리스트
    valid = pd.DataFrame() # validation data를 저장할 리스트
    test = pd.DataFrame() # test data를 저장할 리스트
    for j in range(5):
        train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][:round(len(df) / 5 * 0.9)]], axis=0) # 90% test data
        test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][-round(len(df) / 5 * 0.1):]], axis=0) # 10% test data

    train = train.sample(n=train.shape[0]) # , random_state=1234)
    test = test.sample(n=test.shape[0]) # , random_state=1234)
    
    X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
    X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]
    
    # 교차 검증 정확도를 달성하는 하이퍼파라미터 조합 선택
    best_score = 0
    best_params = None
    for n_estimators in n_est:
        for max_depth in max_d:
            for max_features in max_f:
                for min_samples_split in min_s_s:
                    for min_samples_leaf in min_s_l:
                        for criterion in c:
                            score_sum = 0
                            # 데이터를 5개의 fold로 나누고, Shuffle = True로 지정하여 데이터를 랜덤으로 섞음
                            # k-fold 교차검증을 수행하기 위해 train set과 validation data로 분리
                            for train_index, val_index in KFold(n_splits=5, shuffle=True, random_state=random_state).split(train):
                                X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
                                y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
                                
                                # 오버 샘플링 기법 중 SMOTE 사용
                                smote = SMOTE()
                                # validation data가 아닌, train data에만 smote 기법을 사용
                                X_train_kf_resampled, y_train_kf_resampled = smote.fit_resample(X_train_kf, y_train_kf)
                                
                                clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                                                             min_samples_split=min_samples_split, criterion=criterion, n_jobs=-1)
                                clf.fit(X_train_kf_resampled, y_train_kf_resampled)
                                score_sum += accuracy_score(y_val_kf, clf.predict(X_val_kf)) #5번 교차검증의 정확도의 합
                            score_avg = score_sum / 5 # 5번 교차검증 정확도의 평균
                            if score_avg > best_score: # best_score보다 score_sum이 클 경우 best_score를 score_avg로 업데이트하고 헤딩 파라미터 조합을 best_params에 저장
                                best_score = score_avg # best_score와 score_avg 값이 동일할 경우, False
                                # 각 하이퍼파라미터의 값을 출력
                                best_params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features, 
                                               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'criterion': criterion}
                            # best_score 의 하이퍼파라미터 값 출력
                            print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, max_features: {max_features}, "
                                  f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, criterion: {criterion}",
                                  f" -> accuracy: {score_avg}")
    # 하이퍼 파라미터 값을 갖고 랜덤포레스트 모델 적합                        
    clf = RandomForestClassifier(**best_params)
    clf.fit(X_train_kf, y_train_kf)
    # 하이퍼 파라미터 값을 갖고, test data로 예측한 y_pred 값
    y_pred_rf = clf.predict(X_test)
    # 실제 test값과 예측값 사이의 정확도
    accuracy = accuracy_score(y_test, y_pred_rf)
    results.append(accuracy)

    print(f"Iteration: {i+1}") # 반복 횟수
    print(f"Best params: {best_params}") # 하이퍼파라미터 조합
    print(f"Accuracy on test set: {accuracy}") # accuracy
    print('=='*100)

    test_accuracy_list.append(accuracy) # 반복횟수에 따른 정확도를 저장

result_rf = pd.DataFrame({'Test Accuracy': test_accuracy_list}) # 데이터프레임 형식으로 test_accuracy_list 저장                           
result_rf

n_estimators: 50, max_depth: None, max_features: 4, min_samples_split: 2, min_samples_leaf: 2, criterion: gini  -> accuracy: 0.5368421052631579
Iteration: 1
Best params: {'n_estimators': 50, 'max_depth': None, 'max_features': 4, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Accuracy on test set: 0.8
n_estimators: 50, max_depth: None, max_features: 4, min_samples_split: 2, min_samples_leaf: 2, criterion: gini  -> accuracy: 0.5789473684210525
Iteration: 2
Best params: {'n_estimators': 50, 'max_depth': None, 'max_features': 4, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Accuracy on test set: 0.8
n_estimators: 50, max_depth: None, max_features: 4, min_samples_split: 2, min_samples_leaf: 2, criterion: gini  -> accuracy: 0.4947368421052631
Iteration: 3
Best params: {'n_estimators': 50, 'max_depth': None, 'max_features': 4, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Accuracy on test set: 0.9
n_estimators: 50, max_depth: No

,Test Accuracy
0,0.8
1,0.8
2,0.9
3,0.8
4,0.8
5,0.7
6,0.7
7,0.8
8,0.6
9,0.8
